## Setup

In [ ]:
!pip install transformers[torch] datasets evaluate ml_things -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 38.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 13.6 MB/

In [ ]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"averroussaloom","key":"1dd8a4998933032c172f88dc63f9fc64"}'}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d parulpandey/emotion-dataset

  0% 0.00/715k [00:00<?, ?B/s]
100% 715k/715k [00:00<00:00, 144MB/s]


In [ ]:
!unzip emotion-dataset.zip

Archive:  emotion-dataset.zip
  inflating: test.csv                
  inflating: training.csv            
  inflating: validation.csv          


### import data

In [ ]:
from datasets import load_dataset
import torch

data_folder = "./"

twitter_sentiment = load_dataset("csv", data_files= {
  "train": data_folder + "training.csv",
  "test": data_folder + "test.csv",
  "validation": data_folder + "validation.csv"
})

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [ ]:
device = torch.device("cuda")

In [ ]:
import pandas as pd

df = pd.DataFrame(twitter_sentiment["train"])

In [ ]:
df["label"].unique()

array([0, 3, 2, 5, 4, 1])

## Using BERT

In [ ]:
# load model
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=6)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
def tokenize(data):
  return tokenizer(data["text"], padding="max_length", truncation=True)

tokenized_twitter_sentiment = twitter_sentiment.map(tokenize, batched=True, batch_size=64)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

In [ ]:
import evaluate
import numpy as np
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
  logits, labels = eval_pred
  prediction = np.argmax(logits, axis=-1)
  return metric.compute(predictions=prediction, references=labels)

In [ ]:
tokenized_twitter_sentiment["train"]

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 16000
})

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_twitter_sentiment["train"],
    eval_dataset = tokenized_twitter_sentiment["validation"],
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.222800,0.224487
2,0.154400,0.200238
3,0.106600,0.200333


TrainOutput(global_step=6000, training_loss=0.22343539492289224, metrics={'train_runtime': 5037.6545, 'train_samples_per_second': 9.528, 'train_steps_per_second': 1.191, 'total_flos': 1.2629784231936e+16, 'train_loss': 0.22343539492289224, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.20033316314220428,
 'eval_runtime': 72.6475,
 'eval_samples_per_second': 27.53,
 'eval_steps_per_second': 3.441,
 'epoch': 3.0}

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("/content/standard-bert", num_labels=6)
model

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm

twitter_sentiment_test_dataloader = DataLoader(twitter_sentiment["test"], batch_size = 1, shuffle = False)

true_labels = []
predicted_labels = []


with torch.no_grad():
  for batch in tqdm(twitter_sentiment_test_dataloader, total = len(twitter_sentiment_test_dataloader)):
    tokenized_batch = tokenizer(batch["text"], return_tensors="pt")
    output = model(**tokenized_batch)
    logits = output.logits[0].detach().cpu().numpy()
    prediction = np.argmax(logits)

    predicted_labels.append(prediction)
    true_labels.append(batch['label'].detach().numpy())


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


  0%|          | 0/2000 [00:00<?, ?it/s]

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

evaluation_report = classification_report(true_labels, predicted_labels)
# Show the evaluation report
print(evaluation_report)


              precision    recall  f1-score   support

           0       0.96      0.98      0.97       581
           1       0.95      0.96      0.95       695
           2       0.86      0.79      0.82       159
           3       0.93      0.93      0.93       275
           4       0.90      0.91      0.91       224
           5       0.83      0.68      0.75        66

    accuracy                           0.93      2000
   macro avg       0.91      0.88      0.89      2000
weighted avg       0.93      0.93      0.93      2000



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
model.save_pretrained("./standard-bert")

In [ ]:
!zip -r standard-bert.zip ./standard-bert

updating: standard-bert/ (stored 0%)
  adding: standard-bert/config.json (deflated 54%)
  adding: standard-bert/model.safetensors (deflated 7%)


In [ ]:
!mv /content/standard-bert.zip /content/drive/MyDrive

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Using GPT2

In [ ]:
from torch.utils.data import Dataset
from tqdm.notebook import tqdm
import torch

class TwitterSentimentDataset(Dataset):
  def __init__(self, dataset, use_tokenizer):
    self.texts = dataset["text"]
    self.labels = dataset["label"]
    self.n_examples = len(self.labels)
    return

  def __len__(self):
    return self.n_examples

  def __getitem__(self, item):
    return {"text": self.texts[item], "label": self.labels[item]}

class Gpt2ClassificationCollator(object):
  def __init__(self, use_tokenizer, max_sequence_len = None):
    self.use_tokenizer = use_tokenizer
    self.max_sequence_len = use_tokenizer.model_max_length if max_sequence_len is None else max_sequence_len

    return

  def __call__(self, sequences):
    texts = [sequence["text"] for sequence in sequences]
    labels = [sequence["label"] for sequence in sequences]
    inputs = self.use_tokenizer(text=texts, return_tensors="pt", padding=True, truncation=True, max_length=self.max_sequence_len)
    inputs.update({"labels": torch.tensor(labels)})
    return inputs


def train(dataloader, optimizer_, scheduler_, device_):
  global model

  predictions_labels = []
  true_labels = []
  total_loss = 0

  model.train()

  for batch in tqdm(dataloader, total = len(dataloader)):

    true_labels += batch["labels"].numpy().flatten().tolist()

    batch = {k:v.type(torch.long).to(device_) for k,v in batch.items()}

    model.zero_grad()

    outputs = model(**batch)

    loss, logits = outputs[:2]

    total_loss += loss.item()

    loss.backward()

    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    optimizer_.step()
    scheduler_.step()

    logits = logits.detach().cpu().numpy()

    predictions_labels += logits.argmax(axis = -1).flatten().tolist()

  avg_epoch_loss = total_loss / len(dataloader)

  return true_labels, predictions_labels, avg_epoch_loss

def validate(dataloader, device_):
  global model

  predictions_labels = []
  true_labels = []

  total_loss = 0

  model.eval()

  for batch in tqdm(dataloader, total = len(dataloader)):
    true_labels += batch["labels"].numpy().flatten().tolist()

    batch = {k: v.type(torch.long).to(device_) for k, v in batch.items()}

    with torch.no_grad():
      outputs = model(**batch)

      loss, logits = outputs[:2]

      logits = logits.detach().cpu().numpy()

      total_loss += loss.item()

      predict_content = logits.argmax(axis = -1).flatten().tolist()

      predictions_labels += predict_content

  avg_epoch_loss = total_loss / len(dataloader)

  return true_labels, predictions_labels, avg_epoch_loss

In [ ]:
from transformers import GPT2Config, GPT2Tokenizer, GPT2ForSequenceClassification

device = torch.device("cuda")

model_config = GPT2Config.from_pretrained(pretrained_model_name_or_path="gpt2", num_labels=6)

# we need the next token, so we will pad to left
tokenizer = GPT2Tokenizer.from_pretrained(pretrained_model_name_or_path="gpt2")
tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token

model = GPT2ForSequenceClassification.from_pretrained(pretrained_model_name_or_path="gpt2", config = model_config)
model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = model.config.eos_token_id

model.to(device)


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=6, bias=False)
)

In [ ]:
from torch.utils.data import DataLoader

collator = Gpt2ClassificationCollator(use_tokenizer=tokenizer, max_sequence_len=300)

train_dataset = TwitterSentimentDataset(dataset = twitter_sentiment["train"],
                                        use_tokenizer= tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collator)

valid_dataset = TwitterSentimentDataset(dataset = twitter_sentiment["validation"],
                                        use_tokenizer= tokenizer)
valid_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=False, collate_fn=collator)

test_dataset = TwitterSentimentDataset(dataset = twitter_sentiment["test"],
                                        use_tokenizer= tokenizer)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collator)



In [ ]:
from transformers import AdamW, Trainer, TrainingArguments, get_linear_schedule_with_warmup
from sklearn.metrics import classification_report, accuracy_score
# from ml_things import plot_dict, plot_confusion_matrix, fix_test

epochs = 5

optimizer = AdamW(model.parameters(),
                  lr = 2e-5,
                  eps = 1e-8)

total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)

all_loss = {"train_loss": [], "val_loss": []}
all_acc = {"train_acc": [], "val_acc":[]}

for epoch in tqdm(range(epochs)):
  train_labels, train_predict, train_loss = train(train_dataloader, optimizer, scheduler, device)
  train_acc = accuracy_score(train_labels, train_predict)

  valid_labels, valid_predict, val_loss = validate(valid_dataloader, device)
  val_acc = accuracy_score(valid_labels, valid_predict)
  print("  train_loss: %.5f - val_loss: %.5f - train_acc: %.5f - valid_acc: %.5f"%(train_loss, val_loss, train_acc, val_acc))
  print()

  # Store the loss value for plotting the learning curve.
  all_loss['train_loss'].append(train_loss)
  all_loss['val_loss'].append(val_loss)
  all_acc['train_acc'].append(train_acc)
  all_acc['val_acc'].append(val_acc)

# plot_dict(all_loss, use_xlabel="Epochs", use_ylabel="Value", use_linestyles=['-', '--'])
# plot_dict(all_acc, use_xlabel='Epochs', use_ylabel='Value', use_linestyles=['-', '--'])

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  train_loss: 0.23520 - val_loss: 0.13213 - train_acc: 0.91412 - valid_acc: 0.94063



...

  0%|          | 0/500 [00:00<?, ?it/s]

  train_loss: 0.12159 - val_loss: 0.08713 - train_acc: 0.94419 - valid_acc: 0.95644



  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  train_loss: 0.10977 - val_loss: 0.08314 - train_acc: 0.94819 - valid_acc: 0.95631



In [ ]:
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import numpy as np
twitter_sentiment_test_dataloader = DataLoader(twitter_sentiment["test"], batch_size = 1, shuffle = False)

true_labels = []
predicted_labels = []


with torch.no_grad():
  for batch in tqdm(twitter_sentiment_test_dataloader, total = len(twitter_sentiment_test_dataloader)):
    tokenized_batch = tokenizer(batch["text"], return_tensors="pt")
    tokenized_batch.to(device)

    output = model(**tokenized_batch)
    logits = output.logits[0].detach().cpu().numpy()
    prediction = np.argmax(logits)

    predicted_labels.append(prediction)
    true_labels.append(batch['label'].detach().numpy())

  0%|          | 0/2000 [00:00<?, ?it/s]

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

evaluation_report = classification_report(true_labels, predicted_labels)
# Show the evaluation report
print(evaluation_report)

              precision    recall  f1-score   support

           0       0.96      0.96      0.96       581
           1       0.97      0.92      0.94       695
           2       0.74      0.91      0.81       159
           3       0.96      0.89      0.92       275
           4       0.89      0.93      0.91       224
           5       0.73      0.83      0.78        66

    accuracy                           0.92      2000
   macro avg       0.87      0.91      0.89      2000
weighted avg       0.93      0.92      0.93      2000



In [ ]:
model.save_pretrained("./finetuned-gpt2")

In [ ]:
!zip -r finetuned-gpt2.zip ./finetuned-gpt2
!mv /content/finetuned-gpt2.zip /content/drive/MyDrive

  adding: finetuned-gpt2/ (stored 0%)
  adding: finetuned-gpt2/config.json (deflated 56%)
  adding: finetuned-gpt2/model.safetensors (deflated 7%)
